<a href="https://colab.research.google.com/github/duttagoutam/LJMUMasters/blob/master/TestGenerateCaption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
import matplotlib.pyplot as plt
import nltk
import tensorflow as tf
import keras
import string
import sys, time, os, warnings 
import numpy as np
import pandas as pd
import pydotplus
from pandas import DataFrame
from array import array
from os import listdir
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Activation
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Embedding
from keras.layers.merge import concatenate
from keras.layers.merge import add
from keras.layers.pooling import GlobalMaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu
from numpy import argmax
from pickle import dump
from pickle import load
from keras.utils.vis_utils import model_to_dot

Using TensorFlow backend.


In [0]:
folderName='/content/drive/My Drive/App/storage/Flickr8k_Dataset'
imageDir=folderName+'/Images'

In [0]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [0]:
# load a pre-defined list of photo identifiers
def load_set(filename):
    doc = load_doc(filename)
    dataset = list()
    # process line by line
    for line in doc.split('\n'):
        # skip empty lines
        if len(line) < 1:
            continue
        # get the image identifier
        identifier = line.split('.')[0]
        dataset.append(identifier)
    return set(dataset)

In [0]:
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
    # load document
    doc = load_doc(filename)
    descriptions = dict()
    for line in doc.split('\n'):
        # split line by white space
        tokens = line.split()
        # split id from description
        image_id, image_desc = tokens[0], tokens[1:]
        # skip images not in the set
        if image_id in dataset:
            # create list
            if image_id not in descriptions:
                descriptions[image_id] = list()
            # wrap description in tokens
            desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
            # store
            descriptions[image_id].append(desc)
    return descriptions

In [0]:
# load photo features
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	#print(features)
	return features

In [0]:
# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = list(descriptions.values())
	#print(lines)
	#lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	#print(tokenizer)
	return tokenizer

In [0]:
# Not run Always
# load training dataset (6K)
# filename = folderName+'/Flickr_TextData/Flickr_8k.devImages.txt'
filename = folderName+'/Flickr_TextData/Flickr_8k.trainImages.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions(folderName+'/clean8kdescriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
# save the tokenizer
dump(tokenizer, open(folderName+'/tokenizer8k.pkl', 'wb'))
dump(tokenizer, open(folderName+'/tokenizer8k.txt', 'wb'))
#max_length = maximum_length(train_descriptions)
#print('Description Length: %d' % max_length)

Dataset: 6000
Descriptions: train=6000


In [0]:
def extract_features(filename):
	# load the model
	in_layer = Input(shape=(224, 224, 3))
	model = VGG16(include_top=False, input_tensor=in_layer)
	#print(model.summary())
	# re-structure the model
	#model.layers.pop()
	#model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
	# load the photo
	image = load_img(filename, target_size=(224, 224))
	# convert the image pixels to a numpy array
	image = img_to_array(image)
	# reshape data for the model
	image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
	# prepare the image for the VGG model
	image = preprocess_input(image)
	# get features
	feature = model.predict(image, verbose=0)
	return feature

In [0]:
# map an integer to a word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [0]:
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process
  # 'startseq two dogs playing in the snow endseq'
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		#print("sequence:", sequence)
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		#print("pad sequence:", sequence)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text

In [0]:
# covert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
    all_desc = list()
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

In [0]:
# calculate the length of the description with the most words
def maximum_length(descriptions):
    lines = to_lines(descriptions)
    return max(len(d.split()) for d in lines)

In [0]:
# load test set
filename = folderName+'/Flickr_TextData/Flickr_8k.devImages.txt'
#filename = folderName+'/Flickr_TextData/Flickr_8k.trainImages.txt'
test = load_set(filename)
print('Dataset: %d' % len(test))
# descriptions
test_descriptions = load_clean_descriptions(folderName+'/clean8kdescriptions.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
test_features = load_photo_features(folderName+'/features8k.pkl', test)
print('Photos: test=%d' % len(test_features))
# prepare tokenizer
tokenizer = create_tokenizer(test_descriptions)

tokenizer = load(open(folderName+'/tokenizer8k.pkl', 'rb'))
#print(tokenizer.word_counts)
print(tokenizer.document_count)
#print(tokenizer.word_index)
#print(tokenizer.word_docs)


vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = maximum_length(test_descriptions)
print('Description Length: %d' % max_length)


#print(photo)
# generate description
#for key, desc in train_descriptions.items():
  #newdescription = generate_desc(model, tokenizer, photo[key], max_length)
  #print(newdescription)

Dataset: 1000
Descriptions: test=1000
Photos: test=1000
6000
Vocabulary Size: 29769
Description Length: 31


In [0]:
# load the tokenizer
#tokenizer = load(open(folderName+'/tokenizer8k.pkl', 'rb'))
# pre-define the max sequence length (from training)
max_length = 31
print('Description Length: %d' % max_length)
# load the model
filename = folderName+'/model_0.h5'
model = load_model(filename)
# load and prepare the photograph
#imageFileName = imageDir+'/3484841598_e26ee96aab.jpg'
imageFileName = imageDir+'/10815824_2997e03d76.jpg'
photo = extract_features(imageFileName)

newdescription = generate_desc(model, tokenizer, photo, max_length)
print(newdescription)

In [0]:
# load the tokenizer
picklfile = load(open(folderName+'/tokenizer8k.pkl', 'rb'))
print("Word Docs:",picklfile.word_docs)
print("Word Index Type:", type(tokenizer.word_index))
in_text = 'startseq'
for i in range(max_length):
	# integer encode input sequence
	sequence = tokenizer.texts_to_sequences([in_text])[0]
	print(sequence)
	padsequence = pad_sequences([sequence], maxlen=max_length)
	print(padsequence)

Word Docs: defaultdict(<class 'int'>, {'startseq girl going into wooden building endseq': 1, 'startseq little girl in pink dress going into wooden cabin endseq': 1, 'startseq little girl climbing the stairs to her playhouse endseq': 1, 'startseq child in pink dress is climbing up set of stairs in an entry way endseq': 1, 'startseq little girl climbing into wooden playhouse endseq': 1, 'startseq black dog and white dog with brown spots are staring at each other in the street endseq': 1, 'startseq black dog and tricolored dog playing with each other on the road endseq': 1, 'startseq two dogs on pavement moving toward each other endseq': 1, 'startseq two dogs of different breeds looking at each other on the road endseq': 1, 'startseq black dog and spotted dog are fighting endseq': 1, 'startseq there is girl with pigtails sitting in front of rainbow painting endseq': 1, 'startseq small girl in the grass plays with fingerpaints in front of white canvas with rainbow on it endseq': 1, 'starts

In [0]:
#tokenizernew = Tokenizer()
#tokenizer.fit_on_texts(lines)
fileIndex = tokenizer.word_index
for index in fileIndex.items():
  #print(index[0].split(' ')[0])
  sequence = picklfile.texts_to_sequences(['startseq woman with blonde hair wearing blue tube top is waving on the side of the street endseq'])
  print(sequence)
  break
#print(picklfile.word_docs.keys[0])
in_text = 'startseq'
#sequence = picklfile.texts_to_sequences(picklfile.word_docs)
#print(sequence)

In [0]:
# define 5 documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!']
# create the tokenizer
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(docs)
# summarize what was learned
print(t.word_counts)
print(t.document_count)
print(t.word_index)
print(t.word_docs)

OrderedDict([('well', 1), ('done', 1), ('good', 1), ('work', 2), ('great', 1), ('effort', 1), ('nice', 1), ('excellent', 1)])
5
{'work': 1, 'well': 2, 'done': 3, 'good': 4, 'great': 5, 'effort': 6, 'nice': 7, 'excellent': 8}
defaultdict(<class 'int'>, {'well': 1, 'done': 1, 'good': 1, 'work': 2, 'effort': 1, 'great': 1, 'nice': 1, 'excellent': 1})
